In [1]:
%pwd
!git clone https://github.com/jaywalnut310/vits.git
!python --version
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install numpy
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1
!pip install googletrans

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd

Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (81/81), 3.33 MiB | 7.47 MiB/s, done.
Python 3.10.11
/content/vits
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.34
    Uninstalling Cython-0.29.34:
      Successfully uninstalled Cython-0.29.34
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 87.2 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201378 sha256=17800063bd4bf494f097cab9e3738d841e3ad5b83bbbdb9673bc1b0303

'/content/vits'

In [2]:
import os
import subprocess
import locale
from IPython.display import Audio
import os
import re
import glob
import json
import tempfile
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import numpy as np
import commons
import utils
import argparse
import subprocess
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from scipy.io.wavfile import write
from googletrans import Translator
import soundfile as sf
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
def preprocess_char(text, lang=None):
    """
    Special treatement of characters in certain languages
    """
    print(lang)
    if lang == 'ron':
        text = text.replace("ț", "ţ")
    return text

class TextMapper(object):
    def __init__(self, vocab_file):
        self.symbols = [x.replace("\n", "") for x in open(vocab_file, encoding="utf-8").readlines()]
        self.SPACE_ID = self.symbols.index(" ")
        self._symbol_to_id = {s: i for i, s in enumerate(self.symbols)}
        self._id_to_symbol = {i: s for i, s in enumerate(self.symbols)}

    def text_to_sequence(self, text, cleaner_names):
        '''Converts a string of text to a sequence of IDs corresponding to the symbols in the text.
        Args:
        text: string to convert to a sequence
        cleaner_names: names of the cleaner functions to run the text through
        Returns:
        List of integers corresponding to the symbols in the text
        '''
        sequence = []
        clean_text = text.strip()
        for symbol in clean_text:
            symbol_id = self._symbol_to_id[symbol]
            sequence += [symbol_id]
        return sequence

    def uromanize(self, text, uroman_pl):
        iso = "xxx"
        with tempfile.NamedTemporaryFile() as tf, \
             tempfile.NamedTemporaryFile() as tf2:
            with open(tf.name, "w") as f:
                f.write("\n".join([text]))
            cmd = f"perl " + uroman_pl
            cmd += f" -l {iso} "
            cmd +=  f" < {tf.name} > {tf2.name}"
            os.system(cmd)
            outtexts = []
            with open(tf2.name) as f:
                for line in f:
                    line =  re.sub(r"\s+", " ", line).strip()
                    outtexts.append(line)
            outtext = outtexts[0]
        return outtext

    def get_text(self, text, hps):
        text_norm = self.text_to_sequence(text, hps.data.text_cleaners)
        if hps.data.add_blank:
            text_norm = commons.intersperse(text_norm, 0)
        text_norm = torch.LongTensor(text_norm)
        return text_norm

    def filter_oov(self, text):
        val_chars = self._symbol_to_id
        txt_filt = "".join(list(filter(lambda x: x in val_chars, text)))
        print(f"text after filtering OOV: {txt_filt}")
        return txt_filt

def preprocess_text(txt, text_mapper, hps, uroman_dir=None, lang=None):
    txt = preprocess_char(txt, lang=lang)
    is_uroman = hps.data.training_files.split('.')[-1] == 'uroman'
    if is_uroman:
        with tempfile.TemporaryDirectory() as tmp_dir:
            if uroman_dir is None:
                cmd = f"git clone git@github.com:isi-nlp/uroman.git {tmp_dir}"
                print(cmd)
                subprocess.check_output(cmd, shell=True)
                uroman_dir = tmp_dir
            uroman_pl = os.path.join(uroman_dir, "bin", "uroman.pl")
            print(f"uromanize")
            txt = text_mapper.uromanize(txt, uroman_pl)
            print(f"uroman text: {txt}")
    txt = txt.lower()
    txt = text_mapper.filter_oov(txt)
    return txt

In [4]:
def download(lang, tgt_dir="./"):
  lang_fn, lang_dir = os.path.join(tgt_dir, lang+'.tar.gz'), os.path.join(tgt_dir, lang)
  cmd = ";".join([
        f"wget https://dl.fbaipublicfiles.com/mms/tts/{lang}.tar.gz -O {lang_fn}",
        f"tar zxvf {lang_fn}"
  ])
  print(f"Download model for language: {lang}")
  subprocess.check_output(cmd, shell=True)
  print(f"Model checkpoints in {lang_dir}: {os.listdir(lang_dir)}")
  return lang_dir

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Run inference with {device}")

Run inference with cuda


In [6]:
path_base = '/content/outputs/'


In [7]:
!mkdir '/content/outputs/'

In [8]:
for i in range(100):
  path_new = path_base+'utterance'+str(i+1)
  os.mkdir(str(path_base+'utterance'+str(i+1)))

In [9]:
languages = ['hin','tur','por','hun','ind','eng','spa']

language_to_language = {
    'hin':'hi',
    'tur':'tr',
    'por':'pt',
    'hun':'hu',
    'ind':'id',
    'eng':'en',
    'spa':'es'
  }

empty_lists_dict = {lang: [] for lang in languages}

In [11]:
import pandas as pd

df_new = pd.read_csv('/content/gautam-lang-final.csv')

In [12]:
from tables import path
for lang in languages: 

  LANG = lang
  ckpt_dir = download(LANG)
  vocab_file = f"{ckpt_dir}/vocab.txt"
  config_file = f"{ckpt_dir}/config.json"
  assert os.path.isfile(config_file), f"{config_file} doesn't exist"
  hps = utils.get_hparams_from_file(config_file)
  text_mapper = TextMapper(vocab_file)
  net_g = SynthesizerTrn(
      len(text_mapper.symbols),
      hps.data.filter_length // 2 + 1,
      hps.train.segment_size // hps.data.hop_length,
      **hps.model)
  net_g.to(device)
  _ = net_g.eval()

  g_pth = f"{ckpt_dir}/G_100000.pth"
  print(f"load {g_pth}")

  _ = utils.load_checkpoint(g_pth, net_g, None)

  text_utternaces = df_new[language_to_language[lang]]

  cnt = 1
  for txt in text_utternaces:
    print(f"text: {txt}")
    txt = preprocess_text(txt, text_mapper, hps, lang=LANG)
    stn_tst = text_mapper.get_text(txt, hps)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0).to(device)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
        hyp = net_g.infer(
            x_tst, x_tst_lengths, noise_scale=.667,
            noise_scale_w=0.8, length_scale=1.0
        )[0][0,0].cpu().float().numpy()

    print(f"Generated audio") 
    audio_temp = Audio(hyp, rate=hps.data.sampling_rate)
    path_temp = '/content/outputs/utterance'+str(cnt)+'/'
    filename = lang + '.wav'
    write(path_temp+filename, hps.data.sampling_rate, hyp)
    cnt = cnt + 1

Download model for language: hin
Model checkpoints in ./hin: ['G_100000.pth', 'vocab.txt', 'config.json']
load ./hin/G_100000.pth
text: इस वर ् ष के अंत तक , इस ग ् रह पर लगभग एक अरब लोग ऐसे होंगे जो सक ् रिय रूप से सामाजिक नेटवर ् किंग साइटों का उपयोग करेंगे .
hin
text after filtering OOV: इस वर ् ष के अंत तक  इस ग ् रह पर लगभग एक अरब लोग ऐसे होंगे जो सक ् रिय रूप से सामाजिक नेटवर ् किंग साइटों का उपयोग करेंगे 
Generated audio
text: उन सब में जो एक बात आम है वह है कि वे मरने वाले हैं .
hin
text after filtering OOV: उन सब में जो एक बात आम है वह है कि वे मरने वाले हैं 
Generated audio
text: हालांकि यह कुछ हद तक एक रुग ् ण सोच है , मुझे लगता है कि इसमें कुछ गहरे प ् रभाव हैं जो कि तलाश के लायक हैं .
hin
text after filtering OOV: हालांकि यह कुछ हद तक एक रुग ् ण सोच है  मुझे लगता है कि इसमें कुछ गहरे प ् रभाव हैं जो कि तलाश के लायक हैं 
Generated audio
text: मुझे पहली बार इसका ख ् याल आया डेरेक के.एच. मिलर द ् वारा एक ब ् लॉग पोस ् ट देखने के बाद , जो एक विज ् ञान और प ् रौद ् योगिकी पत ् 

In [16]:
import shutil

# Specify the path to the folder you want to download
folder_path = '/content/outputs'

# Compress the folder into a ZIP file
shutil.make_archive('/content/folder_download', 'zip', folder_path)

# Download the ZIP file
from google.colab import files
files.download('/content/folder_download.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
df = pd.read_csv('/content/gautam.csv')

FileNotFoundError: ignored

In [34]:
len(df)

44